# Ingest News Articles for Scrantenna
This notebook pulls news articles from multiple sources and saves them for further processing.

In [1]:
import requests
import json
import os
from datetime import datetime

# Define constants
API_KEY = 'be93936988fd4df185bd56e8a11125a0'
QUERY = "Scranton"
DATA_DIR = "./data/raw"
NEWS_URL = f"https://newsapi.org/v2/everything?q={QUERY}&apiKey={API_KEY}"

def fetch_news():
    """Fetch news articles from NewsAPI"""
    response = requests.get(NEWS_URL)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch news articles: {response.text}")
    return response.json()

def save_news(articles):
    """Save news articles in JSON format"""
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)
    file_path = os.path.join(DATA_DIR, f"news_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
    with open(file_path, 'w') as f:
        json.dump(articles, f, indent=4)
    print(f"Saved {len(articles)} articles to {file_path}")

# Fetch and save articles
news_data = fetch_news()
save_news(news_data.get('articles', []))

/Users/michaeloboyle/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Saved 100 articles to ./data/raw/news_20240909_181659.json
